In [ ]:
import matplotlib.pyplot as plt
%matplotlib widget
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.pyplot as plt
import chemiscope
from widget_code_input import WidgetCodeInput
from ipywidgets import Textarea
from iam_utils import *
import ase
from ase.io import read, write
import itertools

In [ ]:
#### AVOID folding of output cell 

In [ ]:
%%html

<style>
.output_wrapper, .output {
    height:auto !important;
    max-height:4000px;  /* your desired max-height here */
}
.output_scroll {
    box-shadow:none !important;
    webkit-box-shadow:none !important;
}
</style>

In [ ]:
data_dump = WidgetDataDumper(prefix="ex_02")
display(data_dump)

_Reference textbook / figure credits: Charles Kittel, *Introduction to solid-state physics", Chapter 1_

# Unit cells, fractional coordinates, lattices

A periodic structure is defined by a *lattice* and an a-periodic *repeat unit*. The lattice is a periodic set of points generated by all integer combinations of three *unit cell vectors* $\mathbf{a}_{1,2,3}$, i.e. 

$$
\mathbf{T} = u_1 \mathbf{a}_1 +  u_2 \mathbf{a}_2 +  u_3 \mathbf{a}_3, \quad u_{1,2,3} \in \mathbb{Z}
$$

The repeat unit, or *basis* is defined by the coordinates $\{x_i, y_i, z_i\}$ of a (usually small) number of atoms that sit in arbitrary positions within the unit cell. These are often given in *fractional coordinates* $\{s_{i1}, s_{i2}, s_{i3}\}$, such that the set of all atoms in the crystal is generated by 

$$
(u_1+s_{i1}) \mathbf{a}_1 +  (u_2+s_{i2}) \mathbf{a}_2 + (u_3+s_{i3})
$$

with the $u_{1,2,3}$ ranging over all positive and negative integers, and $i$ ranging over the number of atoms in the basis.

There are 14 types of lattices known as _Bravais lattices_ in three dimensions, that are distinguished by the symmetry group of the lattice points. If you need to refresh your fundamentals of crystallography, and don't remember what a _face centered cubic_ or _body centered cubic_ lattice is, the [wikipedia page](https://en.wikipedia.org/wiki/Bravais_lattice) is excellent. Note also that the crystal structure (lattice+basis) can have more complicated symmetries, forming the 230 [space groups](https://en.wikipedia.org/wiki/Space_group)

Now, consider the structure below. This is just a finite set of atoms, and coordinates are listed individually. You can click on the atoms in the viewer and see its coordinates by clicking on the info panel below the viewer.

In [ ]:
positions = np.array( [[x,y,z] for x,y,z in itertools.product([0, 5, 10, 15],[0, 5, 10, 15],[0, 5, 10, 15]) ])
ase_cube = ase.Atoms("Ga64", positions=positions)

properties = {}
properties["index"] = {
                "target": "atom",
                "values": list(range(1, len(ase_cube)+1)),
            }

properties["coordinates"] = {
                "target": "atom",
                "values": positions,
            }

cs = chemiscope.show([ase_cube], properties=properties, mode="structure",                      
                     environments=chemiscope.all_atomic_environments([ase_cube], cutoff=40),
                     settings={"structure":[{"spaceFilling": False, "environments": {"cutoff": 40}}]}
                    )
display(cs)


<span style="color:blue"> **01** Write a function that returns the lattice vectors and a basis that generates the periodic structure that continues to infinity the motif above. Try also to shift rigidly the basis atoms. Does the resulting crystal change in a significant way? </span>

_NB: the validation code only tests for the primitive cell - you can come up with correct structures that will be marked as incorrect. Trust your own judgment (and look carefully at the visualization)_

In [ ]:
ex01_wci = WidgetCodeInput(
        function_name="sc_lattice_basis", 
        function_parameters="",
        docstring="""
Returns the lattice vectors and the basis (in fractional coordinates) that generates a simple cubic structure.

:return: Three lattice vectors and a list of basis coordinates
""",
            function_body="""
# Write your solution, then click on the button below to update the plotter 
# and check against the reference value

a1 = []
a2 = []
a3 = []
basis = [[]]

return a1, a2, a3, basis
"""
        )

data_dump.register_field("ex01-function", ex01_wci, "function_body")
def ex01_updater():
    a1, a2, a3, basis = ex01_wci.get_function_object()()
    positions = np.asarray(basis)
    h = np.asarray([a1,a2,a3])
    
    structure = ase.Atoms('Ga'+str(len(basis)), positions=positions@h.T, cell=[a1, a2, a3])
    display(chemiscope.show(frames = [structure], mode="structure", 
                            settings={"structure":[{"unitCell":True,"supercell":{"0":3,"1":3,"2":3}}]}
                           ))

def match_lattice(a, b):
    a1, a2, a3, basis = b;
    
    return np.allclose([a1,a2,a3], np.eye(3)*5) and np.asarray(basis).shape==(1,3)
    
"""ref_values = {
        (): ase.Atoms("CNH6", positions=[[ 1.,   -0.,   -0.01],
 [ 2.52, -0.01,  0.  ],
 [ 0.6 ,  1.02, -0.  ],
 [ 0.59, -0.52,  0.88],
 [ 0.6 , -0.51, -0.9 ],
 [ 2.92,  0.5 ,  0.89],
 [ 2.93,  0.51, -0.88],
 [ 2.92, -1.03, -0.  ]]) 
       }, ref_match = match_structure, """    
ex01_wcc = WidgetCodeCheck(ex01_wci, ref_values = { (): ([5,0,0],[0,5,0],[0,0,5],[[0,0,0]]) },
                           ref_match = match_lattice,
                           demo=WidgetUpdater(updater=ex01_updater))    
display(ex01_wcc)

In [ ]:
ex01_txt = Textarea("enter any additional comment", layout=Layout(width="100%"))
data_dump.register_field("ex01-answer", ex01_txt, "value")
display(ex01_txt)

# Primitive cell, supercells, conventional cells

It is always possible to give different descriptions to the same crystal structure. For instance, for a given cell and basis one can always specify a _supercell_ i.e. a multiple of repeat units of the original cell, with a correspondingly larger basis. This basis contains "hidden" symmetries, meaning that it would be possible to describe the same structure with a smaller unit cell. The smallest possible cell is called a _primitive_ (or minimal) cell. 

For instance, the figure below shows three different choices of unit cell for a face-centered cubic lattice. All cells describe the same structure! The first structure is the primitive cell, and the third the conventional cells that reveals more clearly the origin of the name of this lattice.

<img src="figures/fcc-cells.png" width="800"/>

The following visualizer shows a collection of 9 crystal structures, containing either one or two chemical species. Look at them, and try to understand what type of lattice they belong to. You can visualize the environment of each atom within an adjusable cutoff, which may help you appreciate the 3D nature of the structure.

In [ ]:
ase_crystals = read('data/crystals.xyz',":")

properties = {}
properties["lattice vectors a1"] = {
                "target": "structure",
                "values": np.asarray([str(list(a.cell[0])) for a in ase_crystals]),
            }
properties["lattice vectors a2"] = {
                "target": "structure",
                "values": np.asarray([str(list(a.cell[1])) for a in ase_crystals]),
            }
properties["lattice vectors a3"] = {
                "target": "structure",
                "values": np.asarray([str(list(a.cell[2])) for a in ase_crystals]),
            }

properties["basis"] = {
                "target": "atom",
                "values": np.vstack([a.positions for a in ase_crystals]),
            }


cs = chemiscope.show(ase_crystals, properties=properties, mode="structure",                      
                     environments=chemiscope.all_atomic_environments(ase_crystals),
                     settings={"structure":[{"bonds":True, "unitCell":True,"supercell":{"0":3,"1":3,"2":3},
                                            "environments": {"cutoff": 6}}]}                    
                    )

def update_co(change):
    cs.settings={"structure": [{"environments": {"cutoff": pb.value['co']}}]}
pb = WidgetParbox(onchange=update_co, co=(6.,1,12,0.1, r"environment cutoff / Å"))
display(VBox([pb,cs]))


In [ ]:
cs2 = chemiscope.show(ase_crystals, properties=properties, mode="default",                      
                     environments=chemiscope.all_atomic_environments(ase_crystals),
                     settings={"structure":[{"bonds":True, "unitCell":True,"supercell":{"0":3,"1":3,"2":3},
                                            "environments": {"cutoff": 6}}]}                    
                    )
display(cs2)

In [ ]:
cs2.settings = {'map': {'palette' : 'viridis'}, 'structure' }

<span style="color:blue"> **02** Each of the structures above are either face-centered (fcc), body-centered (bcc) or simple cubic (sc). Write down in the box below what is the Bravais lattice for each structure, and the size of the basis used to describe the structure in each frame. </span>

_NB: pay attention: (1) you are asked about the symmetry of the **lattice**: if you just look at the cell, you may be misled; (2) the number of atoms included in the basis depend on the choice of cell._

In [ ]:
ex02_txt = Textarea("structure 1:  lattice: XXX, basis size: YYY\n ....", layout=Layout(width="100%"))
data_dump.register_field("ex02-answer", ex02_txt, "value")
display(ex02_txt)

Using a supercell is not only a way of obtaining a more convenient/intuitive view of a crystal structure. It can also be useful to represent real materials, in which the ideal periodicity of the crystal is broken, e.g. because there are defects, interfaces, or simply disorder. In these cases, one often starts from a small unit cells and explicitly repeats it many times. _This is different from just replicating the unit cell for visualization purposes: the cell is also enlarged, and the coordinates of all atoms inside the unit cell can be adjusted independently_

ASE provides convenient utilities to replicate a structure. If `structure` is an `ase.Atoms` object, then
`structure.repeat((nx, ny, nz))` will replicate the structure the given number of times along each of the axes. 

<span style="color:blue"> **03** Write code to generate a supercell for _fcc_ aluminum, using the number of repeat units as an argument, and a lattice parameter of 4Å. You can use the `repeat` ASE command, but if you feel adventurous you may as well write a replicate function yourself. </span>

_NB: the self-test function can generate false errors because it expects a specific order of the basis atoms. Don't worry too much if it tells you one test has not passed!_

In [ ]:
ex03_wci = WidgetCodeInput(
        function_name="al_supercell", 
        function_parameters="nrep",
        docstring="""
Returns an ASE object describing a supercell built by replicating `nrep` times along each direction a conventional (4-atoms)
fcc unit cell for aluminum. Use a lattice parameter of 4 Å.

:return: The replicated-cell ASE object
""",
            function_body="""
# Write your solution, then click on the button below to update the chemiscope viewer
import ase

al4 = ase.Atoms("Al4", pbc=True)
al4.cell = [ ... ] # lattice parameters of the conventional unit cell
al4.positions = [[, ,], [, ,], [, ,], [, ,]] # positions of the 4 atoms in the conventional fcc cell

al_replicated = ... 

return al_replicated
"""
        )

data_dump.register_field("ex03-function", ex01_wci, "function_body")
def ex03_updater():
    al_multi = ex03_wci.get_function_object()(ex03_wp.value['nrep'])
    display(VBox([ex03_wp,chemiscope.show(frames = [al_multi], mode="structure", 
                            settings={"structure":[{"unitCell":True}]}
                           )]) )

def match_lattice(a, b):    
    return np.allclose(b.cell, a[0]) and (True if a[1] is None else np.allclose(b.positions-b.positions[0], a[1])) 
    
ex03_wp = WidgetParbox(nrep=(2, 1, 4, 1, r"$n_{\mathrm{repeat}}$ (must click on update button!)"))    
ex03_wcc = WidgetCodeCheck(ex03_wci, ref_values = { 
    (1,): ([[4,0,0],[0,4,0],[0,0,4]], [[0,0,0],[2,2,0],[2,0,2],[0,2,2]]),
    (2,): ([[8,0,0],[0,8,0],[0,0,8]], None) 
},
                           ref_match = match_lattice,
                           demo=WidgetUpdater(updater=ex03_updater))
display(ex03_wcc)

# Lattice planes and surfaces 

The general notation used to define lattice planes is somewhat contrived (because it actually relates to the reciprocal lattice!). A plane is defined by three points, so one can define a lattice plane by picking three points that are multiples of the unit cell vectors, $(n_1 \mathbf{a_1}, n_2 \mathbf{a_2}, n_3 \mathbf{a_3})$. The plane is determined by the reciprocals of the $n_{1,2,3}$, multiplied by their least common multiple so as to obtain three integer indices $(h k l)$. For instance, a plane that intercepts the lattice axes with $n_{1,2,3} = (4,3,1)$ has Miller indices $(3,4,12)$. _Only in a cubic lattice_ the plane identified by (h k l) has $h \mathbf{a_1} + k \mathbf{a_2} + l \mathbf{a_3}$ as a plane normal. 

If you need a more detailed discussion, see Chapter 1 of _Kittel_, or the [Wikipedia page on Miller indices](https://en.wikipedia.org/wiki/Miller_index) that has some good figures. 

In the widget below, you can see a large supercell of _fcc_ aluminum. You can select different Miller indices, and see the atoms that belong to one of the lattice planes highlighted in a different color. Experiment a bit and note how e.g. `(2,2,4)` is equivalent to `(1,1,2)`, etc. Note also how the density of atoms on low-index planes is higher than on high-index planes.

In [ ]:
#TODO AG

<span style='color:blue'> **04** Activate the visualization of multiple replicas in the visualizer above. What happens at the edges of the supercell if you pick a `(1,0,0)` plane? And what happens if you pick `(1,1,1)`? </span>

In [ ]:
ex04_txt = Textarea("enter your answer", layout=Layout(width="100%"))
data_dump.register_field("ex04-answer", ex04_txt, "value")
display(ex04_txt)

Now let's create a supercell with an actual surface. While there are tools to do this for more complicated scenarios (see e.g. [here](https://wiki.fysik.dtu.dk/ase/ase/build/surface.html#create-specific-non-common-surfaces) for some examples using ASE), we will do this manually to understand better what is going on. 

If the surface is aligned with one of the lattice parameters, it is sufficient to first replicate the unit cell a number of times, and then artificially enlarge one of the edges of the unit cell. This will leave some empty space between the top and bottom layers of the structure, effectively leaving some atoms in contact with vacuum. 

<img src="figures/slab-100.png" width="400"/>

Note you always create _two_ surfaces, because you are still dealing with a periodic structure. This is usually referred to as a _slab geometry_. Note also that for complicated crystals with a basis, the unit cell can be cut at different positions, so that there can be multiple different surfaces corresponding to the same lattice direction, depending on the position of the cut. 

<span style='color:blue'> **05** Write a function that creates a supercell for _fcc_ aluminum with 4x4x2 replicas along the three directions. Use a conventional unit cell and a lattice parameter of 4Å. Modify the `cell` of the resulting ASE structure to add 20Å of vacuum along the $\mathbf{a}_3$ direction, creating a slab geometry with a surface along `(0,0,1)` </span>

In [ ]:
# TODO AG

Creating a `(1,1,1)` surface is somewhat more complicated, because the conventional unit cell is not aligned properly. One way to create a slab with the appropriate orientation is to create a _primitive_ face-centered cell and to create the slab by _extending_ one of the cell vectors. 

<span style='color:blue'> **06** Write a function that creates a supercell for _fcc_ aluminum, starting with the primitive cell replicated as 8x8x2 along the lattice vectors. Then _multiply_ by four $\mathbf{a}_3$. Visualize the resulting structure</span>

In [ ]:
# TODO AG

<span style="color:blue"> **07** What is the direction of the primitive lattice parameters relative to the conventional _fcc_ index system? Is the surface normal parallel to the $\mathbf{a}_3$ cell vector? 
In order to create an easier to visualize structure, one often builds a unit cell that is orthorhombic and has one axis that is parallel to the surface normal. Can you come up with three low-Miller-indices directions that are mutually orthogonal and could be used to define an appropriate unit cell?
</span>

_NB: The last point is not entirely trivial, think about it but don't get too stressed if you don't manage to create the appropriate cell. If you are *really* into these geometric problems, you can also try to derive an actual unit cell, including also the basis._

In [ ]:
ex07_txt = Textarea("write your answer", layout=Layout(width="100%"))
data_dump.register_field("ex07-answer", ex07_txt, "value")
display(ex07_txt)

# Reciprocal lattice in 2D

From here on, we work exclusively with 2D lattices,

$$
\mathbf{T} = u_1 \mathbf{a}_1 + u_2 \mathbf{a}_2, \quad \mathbf{a}_{1,2} \in \mathbb{R}^2, u_{1,2} \in Z
$$

because they allow for simpler visualization of the core concepts. Most of the concepts we develop and demonstrate, however, apply equally well to actual 3D systems. 

The idea of the reciprocal lattice is deeply linked to the idea of scatterig, and constructive interference. A plane waves $e^{\mathrm{i} \mathbf{k}\cdot \mathbf{x}}$ originating at the origin will be in phase with similar waves originating at all other lattice points if its wavevector $\mathbf{k}$ satisfy the phase relation $\mathbf{k}\cdot \mathbf{T} = 2\pi n$, where $n$ is an integer, for each and every lattice vector. 

We can build a _reciprocal lattice_ that consist of integer combinations of basis vectors $\mathbf{b}_{1,2}$

$$
\mathbf{G} = v_1 \mathbf{b}_1 + v_2 \mathbf{b}_2, \quad \mathbf{b}_{1,2} \in \mathbb{R}^2, v_{1,2} \in Z
$$

such that every reciprocal lattice point is a wavevector that results in a complete in-phase scattering from the direct lattice. To do this, we need relationships to hold between the lattice vectors: $\mathbf{b}_{1,2} \cdot \mathbf{a}_{1,2} = 2\pi$, $\mathbf{b}_{1,2}\cdot \mathbf{a}_{2,1} = 0$. Thus, 
$\mathbf{b}_1$ must be orthogonal to $\mathbf{a}_2$ and $\mathbf{b}_2$ must be orthogonal to $\mathbf{a}_1$.

We can achieve this (including also normalization that ensure the correct scaling $\mathbf{b}_{1,2} \cdot \mathbf{a}_{1,2} = 2\pi$, with the definition

$$ \mathbf{b}_1 = 2\pi\frac{\mathbf{R}\mathbf{a}_2}{\mathbf{a}_1\cdot\mathbf{R}\mathbf{a}_2}, \quad \mathbf{b}_2 = 2\pi\frac{\mathbf{R}\mathbf{a}_1}{\mathbf{a}_2\cdot\mathbf{R}\mathbf{a}_1},\quad\quad\mathbf{R} = \begin{bmatrix}0 & -1\\ 1 & 0\end{bmatrix}$$

where $\mathbf{R}$ is a $\pi/2$ rotation. 

<span style="color:blue"> **08** What are the reciprocal primitive vectors for a 2D rectangular Bravais lattice with primitve vectors $(1,0)$ and $(0,2)$?
</span>

In [ ]:
ex08_txt = Textarea("The reciprocal primitive vectors: b_1 = ... , b_2 = ...", layout=Layout(width="100%"))
data_dump.register_field("ex08-answer", ex08_txt, "value")
display(ex08_txt)

<span style="color:blue"> **09** Write a function that computes the reciprocal lattice vectors given $\mathbf{a}_{1,2}$. Use the demo widget to experiment and get an intuitive understanding of what's going on. 
</span>

In [ ]:
# TODO AG: make an exercise where they build b1, b2 starting from
# a1, a2, and have a demo box that shows two panels, one with the direct lattice
# and one with the reciprocal lattice